In [ ]:
#hide
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

# working directory: drive/MyDrive/spundsteif
%cd '/content/drive/MyDrive/spundsteif'

# install dependency (nbdev version 1 and patch tornado problem)
!pip3 install --upgrade notebook
!pip3 install -e . -q
!pip3 install nbdev==1.2.11 -q

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/spundsteif/spundsteif
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content/drive/MyDrive/spundsteif/spundsteif
/content/drive/MyDrive/spundsteif


In [ ]:
# default_exp pressures

# Library pressures
> This library gives the earth pressure distribution along the sheet piles from the total stress given by the sensors and the piezometer data.  
> The computation of relative pressures compared to a given time is available.  
> Some functions aiming a graphical check of measurements are also available.  
--- 
**Inputs:**  
+ `df_messung`: dataframe of the measurements from sensors
+ `df_piezo.csv`: csv file of the groudwater level
+ `df_excavation.csv`: csv file of the stage of excavation

**Outputs:**  
+ `df_pressures` : total, effective stresses and waterpressure along the sheetpiles.


## Importation des bibliothèques

In [ ]:
#export
import pandas as pd
import numpy
import io
import csv
from datetime import datetime

from google.colab import files

from copy import copy

from datetime import timedelta
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go 

import numpy as np

from spundsteif import sensors

## Test setup

### Test files

In [ ]:
PIEZO_FILE_NAME_TEST = './test/df_piezo_test.csv'
EXCAVATION_FILE_NAME_TEST = './test/df_excavation_stage_test.csv'

# Files needed to get df_measured_pressure
STRUCTURE_DATA = './test/1_structure_file.txt'

TESTS_FILES_NAMES = ['./test/1_data_test_01.ASC',
                     './test/1_data_test_02.txt',
                     './test/1_data_test_03.txt',
                     './test/1_data_test_04.ASC'
                     ]

TEST_HAND = './test/df_hand_test.csv'

### Automatic measured pressure


In [ ]:
df_automatic_pressure = sensors.get_data(list_files_names=TESTS_FILES_NAMES,
                                         structure_data=STRUCTURE_DATA, 
                                         file_name_df_hand=TEST_HAND)

df_automatic_pressure = df_automatic_pressure.loc[:,df_automatic_pressure.columns.str.startswith('EDS')]

/content/drive/MyDrive/spundsteif/spundsteif/sensors.py:394: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_time = df_time.drop(['date'], 1)


In [ ]:
df_automatic_pressure.sample(10, random_state=0)

,EDS_A,EDS_B,EDS_C,EDS_D,EDS_E,EDS_F,EDS_G,EDS_B2
2021-07-12,2242.589478,5.255263e+06,8.988906e+04,NaN,7240.886792,-6.078723e+06,5.736801e+06,NaN
2021-07-09,-2018.012598,3.480844e+06,-4.431066e+04,NaN,12253.052854,-6.144872e+06,7.606129e+05,NaN
2021-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-18,6445.603267,8.499000e+05,2.545558e+06,NaN,-1638.718973,-6.176094e+06,-3.770186e+06,NaN
2021-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-21,-9054.727930,4.607471e+06,2.535880e+06,NaN,-31098.240820,-6.303607e+06,-4.200204e+06,NaN


In [ ]:
df_automatic_pressure.shape

(25, 8)

*Comment of the 28.09.2021*

> We will not go further with automatic pressures since the measurements looks really strange.

## Get raw data

### Read files

In [ ]:
#export
def construct_df_csv(file_name):
  df = pd.read_csv(file_name, delimiter=';')

  #remove empty lines and columns
  df = df.dropna(how='all')
  df = df.dropna(how='all', axis='columns')
  
  return df

In [ ]:
# example
df_piezo = construct_df_csv(PIEZO_FILE_NAME_TEST)
df_piezo

,Datum,Zeit,Tiefe/Top Spundwand [m],Temp [°C]
0,06.07.2021,10:21:00,2.20,14.4
1,07.07.2021,12:00:00,2.15,17.3
2,14.07.2021,10:30:00,1.92,14.5
3,21.07.2021,09:30:00,3.33,13.1
4,28.07.2021,10:00:00,2.11,12.3
5,05.08.2021,09:00:00,2.02,13.0
6,11.08.2021,09:00:00,2.43,14.8
7,18.08.2021,09:30:00,2.40,18.0
8,25.08.2021,11:10:00,2.62,13.6


In [ ]:
# example
df_excavation = construct_df_csv(EXCAVATION_FILE_NAME_TEST)
df_excavation

,Datum,Zeit,Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m]
0,07.07.2021,07:30:00,1.1,1.1,1.0
1,21.07.2021,07:30:00,1.1,1.1,0.5
2,28.07.2021,08:30:00,1.8,1.1,0.5
3,18.08.2021,09:30:00,1.8,1.1,0.5
4,25.08.2021,10:30:00,1.8,1.1,0.5


### Clean data

In [ ]:
#export
def clean(df):
  
  # dtypes
  df['date'] = pd.to_datetime(df['Datum'], format='%d.%m.%Y')

  # remove 'Zeit' and 'Datum'
  df = df.drop(columns=['Datum', 'Zeit'])

  df = df.set_index('date')

  return df

In [ ]:
# example
df_piezo = clean(df_piezo)
df_piezo.dtypes

Tiefe/Top Spundwand [m]    float64
Temp [°C]                  float64
dtype: object

In [ ]:
# example
df_excavation= clean(df_excavation)
df_excavation.dtypes

Aushubstiefe/Verklebtebohlen [m]    float64
Aushubstiefe/Einzelbohlen [m]       float64
Tiefe/Aktivenseite [m]              float64
dtype: object

In [ ]:
# test
for df in df_piezo, df_excavation:
  assert pd.api.types.is_datetime64_any_dtype(df.index)
  assert 'Datum' not in df.columns
  assert 'Zeit' not in df.columns

assert pd.api.types.is_float_dtype(df_excavation['Aushubstiefe/Verklebtebohlen [m]'])
assert pd.api.types.is_float_dtype(df_excavation['Aushubstiefe/Einzelbohlen [m]'])

assert pd.api.types.is_float_dtype(df_piezo['Tiefe/Top Spundwand [m]'])
assert pd.api.types.is_float_dtype(df_piezo['Temp [°C]'])

## Get manual measures

All measures are depth in [m] from the top of the sheet piles

### Merge piezo and excavation

In [ ]:
#export
def merge_piezo_excavation(df_piezo, df_excavation):
  df = df_piezo.join(df_excavation, how='outer')
  return df

In [ ]:
# example
df_manual = merge_piezo_excavation(df_piezo, df_excavation)
df_manual

,Tiefe/Top Spundwand [m],Temp [°C],Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m]
date,,,,,
2021-07-06,2.20,14.4,NaN,NaN,NaN
2021-07-07,2.15,17.3,1.1,1.1,1.0
2021-07-14,1.92,14.5,NaN,NaN,NaN
2021-07-21,3.33,13.1,1.1,1.1,0.5
2021-07-28,2.11,12.3,1.8,1.1,0.5
2021-08-05,2.02,13.0,NaN,NaN,NaN
2021-08-11,2.43,14.8,NaN,NaN,NaN
2021-08-18,2.40,18.0,1.8,1.1,0.5
2021-08-25,2.62,13.6,1.8,1.1,0.5


### Clean manual measures

In [ ]:
#export
def clean_manual_df(df):
  df['GWT [m]'] = df['Tiefe/Top Spundwand [m]']
  df = df.drop(columns=['Tiefe/Top Spundwand [m]'])
  return df

In [ ]:
# example
df_manual = clean_manual_df(df_manual)
df_manual

,Temp [°C],Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m],GWT [m]
date,,,,,
2021-07-06,14.4,NaN,NaN,NaN,2.20
2021-07-07,17.3,1.1,1.1,1.0,2.15
2021-07-14,14.5,NaN,NaN,NaN,1.92
2021-07-21,13.1,1.1,1.1,0.5,3.33
2021-07-28,12.3,1.8,1.1,0.5,2.11
2021-08-05,13.0,NaN,NaN,NaN,2.02
2021-08-11,14.8,NaN,NaN,NaN,2.43
2021-08-18,18.0,1.8,1.1,0.5,2.40
2021-08-25,13.6,1.8,1.1,0.5,2.62


In [ ]:
# test
assert df_manual['GWT [m]']['2021-07-06'] == 2.2

### Get manual dataframe

In [ ]:
#export
def get_manual_data(piezo_file_name, excavation_file_name):
  df_piezo = construct_df_csv(piezo_file_name)
  df_piezo = clean(df_piezo)

  df_excavation = construct_df_csv(excavation_file_name)
  df_excavation = clean(df_excavation)

  df = merge_piezo_excavation(df_piezo, df_excavation)
  df = clean_manual_df(df)

  return df

In [ ]:
# example
df_manual = get_manual_data(PIEZO_FILE_NAME_TEST, EXCAVATION_FILE_NAME_TEST)

fig = go.Figure()
cols = df_manual.columns.drop('Temp [°C]')

for col in cols:
  fig.add_trace(
      go.Scatter(x=df_manual.index, y=df_manual[col], 
                  name=col, 
                  mode='lines+markers', 
                  marker_size=3
      )
  )

# Mise en page
fig.update_layout(
    margin=dict(l=30, r=30, t=50, b=30),
    title='Manual data',
    xaxis_title="Date",
    yaxis_title="Depth in m",
    title_font=dict(family="Rockwell", size=18),
    yaxis = dict(autorange="reversed")
)

fig.show()

## Compute theoretical pressures

Should have a table, for each t

For each sensor location (means depth + active or passive side):

- K (Ko, Ka or Kp)  
- sigma_v  
- u  


Complicated => need the inklino deformation to estimate properly the active/passive or K0 state



In [ ]:
#TODO

## Merge manual and automatic measures

*Comment 28.09.2021*  
> Functions not used as automatic pressure values not realistic  
> 
> TODO: Itegrate EDS_B2 though

#### Merging

In [ ]:
#export
def merge_manual_automatic(df_manual, df_automatic):
    df = df_automatic.join(df_manual, how='outer')
    return df

#### Interpolation and upsampling

In [ ]:
#export 
def resample_data(df):

  ## upsample
  df = df.asfreq(freq='d')

  ## interpolate and fillna
  df = df.interpolate(method='time')
  df = df.fillna(method='bfill')

  return df

## Get pressures

*Remark 28.09.2021*
> Not to use for the moment
> 
> Only use the `get_manual_data` function to proxy the history of loading.

In [ ]:
# export
def get_pressures(df_manual, df_automatic):
  
  df = merge_manual_automatic(df_manual, df_automatic)
  df = resample_data(df)
  return df

In [ ]:
# example
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

e = get_pressures(df_manual, df_automatic_pressure)

cols = e.columns.drop('Temp [°C]')

for col in cols:
    fig.add_trace(
        go.Scatter(x=e.index,
                  y=e[col], 
                  name=col, 
                  mode='lines+markers',
                  marker_size=3 if 'EDS' in col else 5),
        secondary_y=False if 'EDS' in col else True,
    )

# Mise en page
fig.update_layout(
    margin=dict(l=30, r=30, t=50, b=30),
    title='Manual data',
    xaxis_title="Date",
    title_font=dict(family="Rockwell", size=18),
)

   
fig.update_yaxes(title_text="Depth in m", secondary_y=False)
fig.update_yaxes(title_text="Presure in MPa", secondary_y=True)

fig.show()

In [ ]:
# test
assert e.index.is_unique
assert len(e) == 51

assert e['Aushubstiefe/Verklebtebohlen [m]']['2021-07-06'] == 1.1
assert e['Aushubstiefe/Einzelbohlen [m]']['2021-07-06'] == 1.1
assert e['GWT [m]']['2021-07-08'] == 2.117142857142857

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 01_sensors.ipynb.
Converted 02_inclinometers.ipynb.
Converted 03_pressures.ipynb.
Converted 04_deformations.ipynb.
Converted index.ipynb.
